In [7]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import models
from torch import nn

In [2]:
resnet = models.resnet18(pretrained=False)
x= torch.randn(1,3,224,224)
resnet.eval()
torch.onnx.export(resnet,x,'resnet.onnx',input_names=['inputs'],
                  dynamic_axes={"inputs":{0:'bathch',1:'channels'}})

In [23]:
class BasicBlock(nn.Module):
    def __init__(self,ins,outs):
        super().__init__()
        self.conv1 = nn.Conv2d(ins,outs,kernel_size=3,padding=1,stride = 1)
        self.conv2 = nn.Conv2d(outs,outs,kernel_size=3,padding=1,stride = 1)
        self.act =nn.ReLU()
    
    def forward(self,x):
        residual = x
        out = self.conv1(x)
        out = self.act(out)
        out = self.conv2(out)
        return self.act(residual+out)
class BottleBlock(nn.Module):
    def __init__(self,ins,outs):
        super().__init__()
        self.bone = nn.Sequential(nn.Conv2d(ins,outs,kernel_size=3,padding=1,stride = 2),
                                  nn.ReLU(),
                                 nn.Conv2d(outs,outs,kernel_size=3,padding=1,stride = 1),
                                 )
        self.act =nn.ReLU()
        self.conv  =nn.Conv2d(ins,outs,kernel_size=1,padding=0,stride=2)
    def forward(self,x):
        residule = x
        out = self.bone(x)
        return self.act(self.conv(residule)+ out)
class Myresnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem =nn.Sequential(nn.Conv2d(3,64,kernel_size=7,padding=3,stride=2),
                                nn.ReLU(),
                                nn.MaxPool2d(kernel_size=3,stride=2,padding=1))
        self.block =nn.Sequential(BasicBlock(64,64),
                                  BasicBlock(64,64),
                                 BottleBlock(64,128),
                                 BasicBlock(128,128),
                                 BottleBlock(128,256),BasicBlock(256,256),BottleBlock(256,512),
                                 BasicBlock(512,512),)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512,1000)
    def forward(self,x):
        out = self.stem(x)
        out = self.block(out)
        
        out = self.pool(out)
        out =nn.Flatten()(out)

        out = self.fc(out)
        return out
        

In [24]:
testmode = Myresnet()
x =torch.randn(1,3,224,224)
testmode.eval()
torch.onnx.export(testmode,x,'test.onnx',input_names=['inputs'],
                 dynamic_axes={'inputs':{0:'batch',1:'channels'}})